In [10]:
import pandas as pd
import geopandas as gpd
import folium

In [11]:
df = pd.read_feather('../preprocessed_data/2018_9_1_2018_9_30.feather')[['PULocationID','trip_time_cost']]
distance_df = df.groupby('PULocationID').mean().reset_index()
distance_df

,PULocationID,trip_time_cost
0,1,19.470513
1,2,8.333333
2,3,32.310714
3,4,14.109761
4,6,42.400000
...,...,...
252,261,19.013458
253,262,11.813273
254,263,11.145030
255,264,14.341935


In [12]:
sf = gpd.read_file("../raw_data/taxi_zones.shp")
zone = pd.read_csv("../raw_data/taxi+_zone_lookup.csv")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")


In [13]:
gdf = gpd.GeoDataFrame(pd.merge(distance_df, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)
geoJSON = gdf[['LocationID','geometry']].drop_duplicates('LocationID').to_json()

In [14]:
m_trip_time_cost = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON, # geoJSON
    name='choropleth', # name of plot
    data=gdf, # data source
    columns=['LocationID','trip_time_cost'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_time_cost)

m_trip_time_cost.save('../plots/mean_trip_time_cost.html')


In [15]:
# find where is LocationID 2,3

In [16]:
location = pd.DataFrame([[2,100],[3,100]], columns=['LocationID', 'height_light'])


m_trip_time_cost = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON, # geoJSON
    name='choropleth', # name of plot
    data=location, # data source
    columns=['LocationID','height_light'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_time_cost)

m_trip_time_cost.save('../plots/height_light_2_3.html')

In [17]:
df[df["PULocationID"].isin([2,3])]

,PULocationID,trip_time_cost
184444,3,10.983333
563340,3,60.666667
658004,2,8.333333
1490879,3,41.350000
1842578,3,15.316667
2218011,3,19.833333
2375833,3,5.966667
2839766,3,11.033333
3051223,3,69.350000
3359554,3,42.050000


In [18]:
df[df["PULocationID"].isin([2,3])].shape

(15, 2)